In [1]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [39]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [4]:
# This is just a test
df = pd.DataFrame()
fixture = 'fixture.json'
team_home = dirName + '720/720.json'
team_away = dirName + '529/529.json'

with open(fixture) as f:
    my_fixture = json.load(f)
with open(team_home) as f:
    standings_home = json.load(f)
    del standings_home['stats_away']
    my_fixture.update({"team_home": standings_home})
with open(team_away) as f:
    standings_away = json.load(f)
    del standings_home['stats_home']
    my_fixture.update({"team_away": standings_away})
df = df.append(json_normalize(my_fixture))
df.head()

NameError: name 'dirName' is not defined

In [16]:
for col in df.columns: 
    print(col)

goals_home
goals_away
team_home.team.id
team_home.team.name
team_home.rank
team_home.points
team_home.goals_diff
team_home.form
team_home.description
team_away.team.id
team_away.team.name
team_away.rank
team_away.points
team_away.goals_diff
team_away.form
team_away.description
team_away.stats_home.played
team_away.stats_home.wins
team_away.stats_home.draws
team_away.stats_home.lose
team_away.stats_home.goals_for
team_away.stats_home.avg_goals_for
team_away.stats_home.goals_against
team_away.stats_home.avg_goals_against
team_away.stats_home.clean_sheet
team_away.stats_home.failed_to_score
team_away.stats_home.streaks.wins
team_away.stats_home.streaks.draws
team_away.stats_home.streaks.loses
team_away.stats_home.streaks.best_win
team_away.stats_home.streaks.best_lose
team_away.stats_home.streaks.best_goals_for
team_away.stats_home.streaks.best_goals_against
team_away.stats_away.played
team_away.stats_away.wins
team_away.stats_away.draws
team_away.stats_away.lose
team_away.stats_away.goal

In [4]:
league = '140'
season = '2019'

In [5]:
# League directory
dirName = league + '/' + season + '/' 

In [40]:
# Multi League directory
country = 'GB'
season = '2019'
dirCountry = '../' + country + '/'

In [41]:
# Historicity: multi season directory
year_window = 1
start_season = 2019
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2019]


In [35]:
prev_stand = '_36.json'
curr_stand = '_37.json'

In [84]:
df = pd.DataFrame()
standings = None
curr_standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings
visited_standings = {}
# This will help to map to current standings
current_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if league == "teams.json":
        continue
    if ".csv" in league:
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'
            
            # Gets standings from a team if they have not being added
            if team_id not in visited_standings:
                json_standings = dirTeam + team_id + prev_stand
                with open(json_standings) as f:
                    standings = json.load(f)
                    visited_standings[team_id] = standings
            else:
                standings = visited_standings[team_id]
                
            # Gets last standings from a team if they have not being added
            if team_id not in current_standings:
                json_standings = dirTeam + team_id + curr_stand
                with open(json_standings) as f:
                    curr_standings = json.load(f)
                    current_standings[team_id] = curr_standings
            else:
                curr_standings = current_standings[team_id]

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        # Gets away current standings if fixtures is set to null
                        if fixture["goals_away"] is None:
                            if away_team in current_standings:
                                away_team_stands = current_standings[away_team]
                            else:
                                # Get standings from away_team
                                fileAwayTeam = dirName + away_team + '/' + away_team + curr_stand
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    current_standings[away_team] = away_team_stands
                            # Changes if game to be played
                            standings = current_standings[team_id]
                        else:
                            if away_team in visited_standings:
                                away_team_stands = visited_standings[away_team]
                            else:
                                # Get standings from away_team
                                fileAwayTeam = dirName + away_team + '/' + away_team + prev_stand
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[away_team] = away_team_stands
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    if fixture["team_home"]["team"]["name"] == 'Norwich':
                        print(fixture)
                    df = df.append(json_normalize(fixture))
                else:
                    continue

{'team_home': {'team': {'id': '71', 'name': 'Norwich'}, 'rank': 20, 'points': 21, 'goals_diff': -44, 'form': 'LLLLL', 'description': 'Relegation - Championship', 'stats_home': {'played': 19, 'wins': 4, 'draws': 3, 'lose': 12, 'goals_for': 19.0, 'avg_goals_for': 1.0, 'goals_against': 37.0, 'avg_goals_against': 1.9, 'clean_sheet': 2, 'failed_to_score': 7, 'streaks': {'wins': 1, 'draws': 2, 'loses': 3, 'best_win': '3-1', 'best_lose': '1-5', 'best_goals_for': 3, 'best_goals_against': 5}}, 'stats_away': {'played': 18, 'wins': 1, 'draws': 3, 'lose': 14, 'goals_for': 7.0, 'avg_goals_for': 0.4, 'goals_against': 33.0, 'avg_goals_against': 1.8, 'clean_sheet': 3, 'failed_to_score': 12, 'streaks': {'wins': 1, 'draws': 2, 'loses': 3, 'best_win': '0-2', 'best_lose': '0-2', 'best_goals_for': 2, 'best_goals_against': 4}}}, 'team_away': {'team': {'id': '52', 'name': 'Crystal Palace'}, 'rank': 14, 'points': 42, 'goals_diff': -17, 'form': 'LLLLL', 'description': None, 'stats_home': {'played': 18, 'wins':

{'team_home': {'team': {'id': '71', 'name': 'Norwich'}, 'rank': 20, 'points': 21, 'goals_diff': -44, 'form': 'LLLLL', 'description': 'Relegation - Championship', 'stats_home': {'played': 19, 'wins': 4, 'draws': 3, 'lose': 12, 'goals_for': 19.0, 'avg_goals_for': 1.0, 'goals_against': 37.0, 'avg_goals_against': 1.9, 'clean_sheet': 2, 'failed_to_score': 7, 'streaks': {'wins': 1, 'draws': 2, 'loses': 3, 'best_win': '3-1', 'best_lose': '1-5', 'best_goals_for': 3, 'best_goals_against': 5}}, 'stats_away': {'played': 18, 'wins': 1, 'draws': 3, 'lose': 14, 'goals_for': 7.0, 'avg_goals_for': 0.4, 'goals_against': 33.0, 'avg_goals_against': 1.8, 'clean_sheet': 3, 'failed_to_score': 12, 'streaks': {'wins': 1, 'draws': 2, 'loses': 3, 'best_win': '0-2', 'best_lose': '0-2', 'best_goals_for': 2, 'best_goals_against': 4}}}, 'team_away': {'team': {'id': '48', 'name': 'West Ham'}, 'rank': 15, 'points': 37, 'goals_diff': -13, 'form': 'WWLDW', 'description': None, 'stats_home': {'played': 18, 'wins': 6, 'd

{'team_home': {'team': {'id': '71', 'name': 'Norwich'}, 'rank': 20, 'points': 21, 'goals_diff': -44, 'form': 'LLLLL', 'description': 'Relegation - Championship', 'stats_home': {'played': 19, 'wins': 4, 'draws': 3, 'lose': 12, 'goals_for': 19.0, 'avg_goals_for': 1.0, 'goals_against': 37.0, 'avg_goals_against': 1.9, 'clean_sheet': 2, 'failed_to_score': 7, 'streaks': {'wins': 1, 'draws': 2, 'loses': 3, 'best_win': '3-1', 'best_lose': '1-5', 'best_goals_for': 3, 'best_goals_against': 5}}, 'stats_away': {'played': 18, 'wins': 1, 'draws': 3, 'lose': 14, 'goals_for': 7.0, 'avg_goals_for': 0.4, 'goals_against': 33.0, 'avg_goals_against': 1.8, 'clean_sheet': 3, 'failed_to_score': 12, 'streaks': {'wins': 1, 'draws': 2, 'loses': 3, 'best_win': '0-2', 'best_lose': '0-2', 'best_goals_for': 2, 'best_goals_against': 4}}}, 'team_away': {'team': {'id': '38', 'name': 'Watford'}, 'rank': 17, 'points': 34, 'goals_diff': -23, 'form': 'LWWLL', 'description': None, 'stats_home': {'played': 18, 'wins': 6, 'dr

In [76]:
df.head(10)

,goals_home,goals_away,league,season,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,team_home.goals_diff,team_home.form,...,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away,team_home.group,team_away.group
0,1,3,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,1,None,5,341.0,234.0,69%,NaN,NaN,NaN,NaN
0,2,0,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,3,None,1,539.0,435.0,81%,NaN,NaN,NaN,NaN
0,2,2,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,4,None,1,281.0,177.0,63%,NaN,NaN,NaN,NaN
0,0,0,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,3,1,5,470.0,391.0,83%,NaN,NaN,NaN,NaN
0,1,2,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,2,None,6,295.0,208.0,71%,NaN,NaN,NaN,NaN
0,2,3,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,None,None,3,386.0,292.0,76%,NaN,NaN,NaN,NaN
0,1,0,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,3,None,1,508.0,424.0,83%,NaN,NaN,NaN,NaN
0,2,1,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,2,None,3,314.0,214.0,68%,NaN,NaN,NaN,NaN
0,1,6,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,1,None,None,877.0,809.0,92%,NaN,NaN,NaN,NaN
0,1,0,39,2019,66,Aston Villa,19,31,-27,DWLLL,...,4,None,2,595.0,522.0,88%,NaN,NaN,NaN,NaN


In [77]:
norwich = df[(df["team_home.team.name"]=='Norwich')]
print(norwich["team_home.stats_home.played"])

0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
0    19
Name: team_home.stats_home.played, dtype: int64


In [23]:
just_numbers = []
for fixture in visited_fixtures:
    just_numbers.append(int(fixture.strip('.json')))
print(just_numbers)

[157186, 157036, 157076, 157056, 157026, 157266, 157196, 157246, 157226, 157376, 157356, 157117, 157286, 157137, 157096, 157327, 157166, 157347, 157306, 157169, 157391, 157309, 157109, 157030, 157071, 157289, 157270, 157051, 157089, 157209, 157141, 157121, 157239, 157331, 157219, 157189, 157371, 157351, 157305, 157165, 157325, 157017, 157095, 157345, 157285, 157115, 157355, 157375, 157135, 157225, 157195, 157245, 157055, 157075, 157035, 157255, 157185, 157057, 157348, 157236, 157037, 157328, 157187, 157216, 157118, 157138, 157206, 157267, 157027, 157287, 157086, 157357, 157377, 157106, 157307, 157167, 157190, 157240, 157372, 157220, 157261, 157210, 157090, 157340, 157052, 157290, 157072, 157320, 157110, 157130, 157170, 157023, 157310, 157392, 157150, 157201, 157260, 157149, 157129, 157319, 157231, 157339, 157379, 157181, 157251, 157359, 157024, 157301, 157101, 157160, 157281, 157059, 157039, 157081, 157295, 157365, 157105, 157175, 157155, 157085, 157136, 157065, 157045, 157116, 157205,

In [66]:
df.shape

(395, 122)

In [67]:
# csv to save
file_title = 'premier_38.csv'
csv_file = dirCountry + file_title

In [68]:
df.to_csv(csv_file)

In [51]:
os.listdir(dirName)

['533',
 '798',
 '532',
 '.DS_Store',
 '543',
 '715',
 '529',
 '545',
 '542',
 '537',
 '530',
 '539',
 '538',
 '531',
 '536',
 '540',
 '541',
 '546',
 '548',
 '720',
 '727']